In [1]:
import torch
from torch import nn, optim, autograd as grad
from torch.utils.data import DataLoader, random_split, dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from AddClass import *
import time
#import re

In [2]:
Embed_dim = 300
batch_size = 64
n_work = 2
start_time = time.time()

In [3]:
class Rnn(nn.Module):
    
    def __init__(self):
        super(Rnn, self).__init__()
        #self.rnnL1 = nn.RNN(input_size=Embed_dim ,hidden_size=1024,
        #        nonlinearity= 'tanh' ,num_layers=num_layers, batch_first= True)
        self.rnnL1 = nn.RNNCell(input_size=Embed_dim ,hidden_size=1024)
        self.rnnL2 = nn.RNNCell(input_size=1024 ,hidden_size=256)
        self.linL3 = nn.Linear(256,5)
        self.relu = nn.ReLU()
        
        self.dropout = nn.Dropout(0.5)
        self.softmax = nn.Softmax()
    def forward(self,x):
        #x,hidden =self.rnnL1(x)
        x=self.dropout(self.rnnL1(x))
        x=self.rnnL2(x)
        x=self.relu(self.linL3(x))
        x=self.softmax(x)
        
        return x
        

Dataset call section

In [4]:
dir_path = r"C:\Users\orian\OneDrive\שולחן העבודה\Coding Files 2\Datasets"
dataset = pd.read_csv(dir_path+r"\chat gpt reviews\ChatGPT_Reviews.csv")
emb_dict = pd.read_csv(dir_path+r"\embedding dictionary\dict(2).csv") # A dictionary of words to be embedded

print(emb_dict.keys())

# Create a dataset dictionary of reviews and ratings
part_ds = [dataset['Review'], torch.tensor(dataset['Ratings'])]

temp_part_ds = []
# 
part_ds = list(zip(part_ds[0], part_ds[1]-1))

train_len = round(len(part_ds)*0.9)
train_ds_csv, test_ds_csv = random_split(
    dataset=part_ds,lengths=[train_len, len(part_ds)-train_len])
train_data = DataLoader(train_ds_csv, shuffle= True, 
                        num_workers= n_work, batch_size=batch_size)

Index(['word'], dtype='object')


Embedding Section

In [5]:
word_dict = emb_dict['word'].tolist()
word_dict = set(word_dict)

# Separate the words in the reviews and make sure they don't repeat themselves
# (They are going to be added to the embeddings)
review_set = set(organize_text(dataset['Review'].astype(str)))

print('10/10' in review_set)

combined_words = word_dict.union(review_set)
# Dictionary to map words to indices
word_to_index = {key: val for val, key in enumerate(combined_words)}
embedding_layer = nn.Embedding(num_embeddings=len(word_to_index)
                               , embedding_dim=Embed_dim)

True


Single Epoch Training:

In [ ]:
def train_one_epoch(rnn, train_ds, optimizer, dev, FILE_PATH, epoch_iter
                 # Should add stuff that's required
):
    iter = 0
    rnn = rnn.to(dev)
    
    # Training loop part
    for batch in train_ds:
        sample_x, sample_y = batch 
        loss = 0 # Reset loss for each batch
        grad_tmp = None # Initialize gradient measure

        for i in range(len(sample_x)): # Iterate over the batch

            #rnn = rnn.to(dev) # Added due to errors 
            # (The device probably resets each iteration for some reason should remain in the loop)
            
            # ---PreProcessing---
            targetY = torch.nn.functional.one_hot(
                torch.tensor(sample_y[i]), num_classes=5)
            targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
            targetY = torch.squeeze(targetY).to(dev)
            

            # Pre processing the input words to make sense out of them
            input_words = tupleToArray(sample_x[i])
            input_words = organize_text(input_words)

            #Skipping problematic words
            try:
                embed_var = torch.LongTensor([word_to_index[word] for word in input_words])
                embed_var = embedding_layer(embed_var).to(dev)
                
                # ---ForwardPropagation---
                tmp_fwd = rnn.forward(embed_var).to(dev)
                pred = tmp_fwd[len(tmp_fwd)-1].flatten().to(dev)
                
                loss_func = nn.CrossEntropyLoss()
                loss = loss + loss_func(pred, targetY)
                
            except KeyError:
                pass

            if iter%100==0:
                torch.save(rnn.cpu(), FILE_PATH)
                print(sample_x[i],"\n",sample_y[i])
                if grad_tmp != None:
                    print(iter,"({})".format(epoch_iter+1), " | ",grad_tmp.sum()**2/len(grad_tmp),
                        " | time elapsed: ", time.time() - start_time)

            optimizer.zero_grad()
            iter+=1
        
        print("Backpropagated")
        grad_tmp = rnn.linL3.weight.grad
        
        # ---BackPropagation---
        loss.backward()
        optimizer.step()
    

Main Section

In [ ]:
if __name__ == "__main__":
    if torch.cuda.is_available() : device = "cuda:0"
    else : device = "cpu"
    device = torch.device(device)
        
    rnn = Rnn().to(device)
    optimizer = optim.Adam(rnn.parameters(), lr=0.001,
                           weight_decay=1e-4, amsgrad=True)
    start_time = time.time() #Reset start time
    for epoch in range(1):  # You can set the number of epochs as needed
        train_one_epoch(rnn, train_data, optimizer=optimizer, dev=device, FILE_PATH="model.pth", epoch_iter=epoch)
    

C:\Users\orian\AppData\Local\Temp\ipykernel_22116\3599331313.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(sample_y[i]), num_classes=5)
C:\Users\orian\AppData\Local\Temp\ipykernel_22116\3599331313.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targetY = torch.tensor(targetY, dtype=torch.float32, requires_grad=True)
C:\Users\orian\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Cool 
 tensor(4)
Backpropagated
I like it a lot its can talk if you don't have any company and assignment solve by just talking 
 tensor(4)
Backpropagated
